# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [24]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

If cvFitLine() returns normalized vector (vx,vy) and point (x0,y0), then the equation of the line is

(x,y) = (x0,y0) + t*(vx,vy)

where t runs from −∞ to +∞.

Just draw a big line instead of solving for the boundaries. eg:

cv.Line(img, (x0-m*vx[0], y0-m*vy[0]), (x0+m*vx[0], y0+m*vy[0]), (0,0,0))

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [25]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    filtered_lines = []
    Min_angle = math.tan(math.pi/9) # 20 degrees
    Max_angle = math.tan(math.pi*7/18)  # 70 degrees
    print ("Min_angle",Min_angle)
    print ("Max_angle",Max_angle)
    # Filter lines that have slope near horizontal or near vertical
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1 !=x2:
                slope= ((y2-y1)/(x2-x1))
                if abs(slope) > Min_angle and abs(slope) < Max_angle:
                    filtered_lines.append([x1,y1,x2,y2,slope])
                    
                
                
    
    
    # Sort the lines by their slopes.
    lines = sorted(filtered_lines,key=lambda coordinates: coordinates[4])
    
    #print ("lines ",lines)

    
#     for line in lines:
#         print ((line[3]-line[1])/(line[2]-line[0]))
#         print ("\n")
    Left_Lines=[]
    Right_Lines=[]
    for line in lines:
        print (line)
        x1,y1,x2,y2,slope = line
        if slope <0:
            # Left Line
            color=[0, 0, 255]
            Left_Lines.append((x1,y1))
            Left_Lines.append((x2,y2))
            
        else:
            # Right Line
            color=[255, 0, 0]
            Right_Lines.append((x1,y1))
            Right_Lines.append((x2,y2))
#             print ("\n")
#         cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    param = 0
    reps = 0.01 
    aeps = 0.01
    vx_left, vy_left, cx_left, cy_left  = cv2.fitLine(np.array(Left_Lines), cv2.DIST_L2, param, reps, aeps) 
    vx_right, vy_right, cx_right, cy_right  = cv2.fitLine(np.array(Right_Lines), cv2.DIST_L2, param, reps, aeps)
    print ("vx_left, vy_left, cx_left, cy_left ", vx_left, vy_left, cx_left, cy_left)
    print ("vx_right , vy_right , cx_right , cy_right  ", vx_right , vy_right , cx_right , cy_right )
    m_left = 100
    m_right= 100
    y_max = int(imshape[0])
    print ("y_max ",y_max)
    y_min = int((imshape[0]/2)+(imshape[0]/8))
    m_left_bottom = float(y_max - cy_left)/float(vy_left)  
    print ("m_left_bottom ",m_left_bottom)
    m_left_up = float(y_min - cy_left)/float(vy_left)
    
    m_right_bottom = float(y_max - cy_right)/float(vy_right)  
    print ("m_right_bottom ",m_right_bottom)
    m_right_up = float(y_min - cy_right)/float(vy_right)
    
    point_x1_left = int(cx_left+m_left_bottom*vx_left)
    point_y1_left = int(cy_left+m_left_bottom*vy_left)
    point_x2_left = int(cx_left+m_left_up*vx_left)
    point_y2_left = int(cy_left+m_left_up*vy_left)
    
    point_x1_right = int(cx_right+m_right_bottom*vx_right)
    point_y1_right = int(cy_right+m_right_bottom*vy_right)
    point_x2_right = int(cx_right+m_right_up*vx_right)
    point_y2_right = int(cy_right+m_right_up*vy_right)
    print ("point_x1_left, point_y1_left, point_x2_left, point_y2_left ", point_x1_left, point_y1_left, point_x2_left, point_y2_left)
    print ("point_x1_right, point_y1_right, point_x2_right, point_y2_right ", point_x1_right, point_y1_right, point_x2_right, point_y2_right)
    cv2.line(img, (point_x1_left, point_y1_left), (point_x2_left, point_y2_left), (255,0,0),3)
    cv2.line(img, (point_x1_right, point_y1_right), (point_x2_right, point_y2_right), (0,0,255),3)
            
#     print ("Left Lines: ")
#     print (Left_Lines)
#     print ("Right Lines: ")
#     print (Right_Lines)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [26]:
import os
os.listdir("test_images/")

['solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [27]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

for imageFile in os.listdir("test_images/"):
    #reading in an image
    imageName = os.path.join("test_images",imageFile)
    image = cv2.imread(imageName)
    #printing out some stats and plotting
    print('This image is:', imageName, 'with dimesions:', image.shape)
    # Convert image to grayscale
    gray = grayscale(image)


    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray,5)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Define Region of Interest
    imshape = edges.shape
    offset_x = imshape[1]/10
    offset_y = imshape[0]/8
    vertices = np.array([[(0,imshape[0]),(imshape[1]/2-offset_x, imshape[0]/2+offset_y), (imshape[1]/2+offset_x, imshape[0]/2+offset_y), (imshape[1],imshape[0])]], dtype=np.int32)
    ROI = region_of_interest(edges,vertices)
    # cv2.imwrite("output_ROI.jpg",ROI) 

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    line_image = hough_lines(ROI, rho, theta, threshold, min_line_len, max_line_gap)

    final_image =  weighted_img(line_image,image,α=0.4,β=6.0)

    #lanesFilename = os.path.join('output',imageFile.split('.')[0]+'_outputLanes.jpg')
    finalFilename = os.path.join('test_images',imageFile.split('.')[0]+'_outputFinal.jpg')
    #cv2.imwrite(lanesFilename,line_image)
    cv2.imwrite(finalFilename,final_image)

This image is: test_images/solidWhiteRight.jpg with dimesions: (540, 960, 3)
Min_angle 0.36397023426620234
Max_angle 2.7474774194546216
[393, 372, 439, 339, -0.71739130434782605]
[387, 371, 435, 337, -0.70833333333333337]
[390, 369, 436, 337, -0.69565217391304346]
[318, 425, 362, 395, -0.68181818181818177]
[389, 372, 439, 338, -0.68000000000000005]
[310, 423, 356, 392, -0.67391304347826086]
[316, 425, 357, 398, -0.65853658536585369]
[547, 347, 609, 385, 0.61290322580645162]
[530, 337, 853, 539, 0.62538699690402477]
[660, 426, 718, 463, 0.63793103448275867]
[772, 499, 822, 531, 0.64000000000000001]
[787, 496, 840, 530, 0.64150943396226412]
[525, 337, 772, 498, 0.65182186234817818]
[721, 466, 832, 539, 0.65765765765765771]
vx_left, vy_left, cx_left, cy_left  [ 0.81567657] [-0.57850814] [ 380.5] [ 378.07144165]
vx_right , vy_right , cx_right , cy_right   [ 0.84107566] [ 0.54091752] [ 713.42858887] [ 456.64285278]
y_max  540
m_left_bottom  -279.9071396396118
m_right_bottom  154.10325010550

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))